In [1]:
!pip install --user flask==0.12 \
                dash==0.21.0 \
                dash-core-components==0.21.1\
                dash-html-components==0.9.0\
                dash-renderer==0.12.1\
                dash-table-experiments==0.6.0

     |████████████████████████████████| 82 kB 2.4 MB/s eta 0:00:011
     |████████████████████████████████| 1.9 MB 52.4 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 8.8 MB/s  eta 0:00:01
     |████████████████████████████████| 160 kB 92.9 MB/s eta 0:00:01
     |████████████████████████████████| 738 kB 80.3 MB/s eta 0:00:01
     |████████████████████████████████| 27.7 MB 52.7 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-0.21.0-py3-none-any.whl size=17577 sha256=158b11275e2a573f05012b7acbd554fcee8bf4bdbf764d38749df6f3dfa8f543
  Stored in directory: /home/ubuntu/.cache/pip/wheels/cc/06/1f/431467780583b82f0dbdfceacf6d7ac9de1b2b90132d359ca6
  Created wheel for dash-core-components: filename=dash_core_components-0.21.1-py3-none-any.whl size=1962668 sha256=994ed83db41674c6b64b75b13337a3e8f310db97546ff6b0be3c3a82aa71d3c2
  Stored in directory: /home/ubuntu/.cache/pip/wheels/76/85/a0/20c5f901bf970e79115d46888171be37cd4e24f4a4c667c693
  Created wheel for dash-html-c

### source
https://github.com/plotly/dash-table-experiments/blob/master/usage.py#L122

In [8]:
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
import json
import pandas as pd
import numpy as np
import plotly
import os

In [9]:
#server = Flask(__name__)
#app = dash.Dash(server=server)

portID = '8883'
runID = os.environ['DOMINO_RUN_ID']
user = os.environ['DOMINO_PROJECT_OWNER']
project = os.environ['DOMINO_PROJECT_NAME']

app_path = '<Domino url>'+ user + '/' + project +'/notebookSession/' + runID + '/proxy/' + portID
assets_path = app_path + '/assets/'
print(app_path)

#set the external path to the assets folder
app = dash.Dash(assets_external_path = assets_path)
#app = dash.Dash()

app.scripts.config.serve_locally = True
app.css.config.serve_locally = True    
app = dash.Dash()
 
app.scripts.config.serve_locally = True
# app.css.config.serve_locally = True
 
################################################################
# Configure path for dependencies. This is required for Domino.
#Learn more about Dash on Domino https://blog.dominodatalab.com/building-domino-web-app-dash/
#del app.config._read_only["requests_pathname_prefix"]

app.config.routes_pathname_prefix=''+portID+'/'
app.config.requests_pathname_prefix=''+portID+'/'
#################################################################


<Domino url>alireza_mounesisohi/quick-start-project/notebookSession/623c1a458f671737cbf0cd9a/proxy/8883


In [10]:
DF_WALMART = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/1962_2006_walmart_store_openings.csv')

DF_GAPMINDER = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
)
DF_GAPMINDER = DF_GAPMINDER[DF_GAPMINDER['year'] == 2007]
DF_GAPMINDER.loc[0:20]

DF_SIMPLE = pd.DataFrame({
    'x': ['A', 'B', 'C', 'D', 'E', 'F'],
    'y': [4, 3, 1, 2, 3, 6],
    'z': ['a', 'b', 'c', 'a', 'b', 'c']
})

ROWS = [
    {'a': 'AA', 'b': 1},
    {'a': 'AB', 'b': 2},
    {'a': 'BB', 'b': 3},
    {'a': 'BC', 'b': 4},
    {'a': 'CC', 'b': 5},
    {'a': 'CD', 'b': 6}
]


app.layout = html.Div([
    html.H4('Gapminder DataTable'),
    dt.DataTable(
        rows=DF_GAPMINDER.to_dict('records'),

        # optional - sets the order of columns
        columns=sorted(DF_GAPMINDER.columns),

        row_selectable=True,
        filterable=True,
        sortable=True,
        selected_row_indices=[],
        id='datatable-gapminder'
    ),
    html.Div(id='selected-indexes'),
    dcc.Graph(
        id='graph-gapminder'
    ),
], className="container")

In [11]:
@app.callback(
    Output('datatable-gapminder', 'selected_row_indices'),
    [Input('graph-gapminder', 'clickData')],
    [State('datatable-gapminder', 'selected_row_indices')])
def update_selected_row_indices(clickData, selected_row_indices):
    if clickData:
        for point in clickData['points']:
            if point['pointNumber'] in selected_row_indices:
                selected_row_indices.remove(point['pointNumber'])
            else:
                selected_row_indices.append(point['pointNumber'])
    return selected_row_indices


@app.callback(
    Output('graph-gapminder', 'figure'),
    [Input('datatable-gapminder', 'rows'),
     Input('datatable-gapminder', 'selected_row_indices')])
def update_figure(rows, selected_row_indices):
    dff = pd.DataFrame(rows)
    fig = plotly.tools.make_subplots(
        rows=3, cols=1,
        subplot_titles=('Life Expectancy', 'GDP Per Capita', 'Population',),
        shared_xaxes=True)
    marker = {'color': ['#0074D9']*len(dff)}
    for i in (selected_row_indices or []):
        marker['color'][i] = '#FF851B'
    fig.append_trace({
        'x': dff['country'],
        'y': dff['lifeExp'],
        'type': 'bar',
        'marker': marker
    }, 1, 1)
    fig.append_trace({
        'x': dff['country'],
        'y': dff['gdpPercap'],
        'type': 'bar',
        'marker': marker
    }, 2, 1)
    fig.append_trace({
        'x': dff['country'],
        'y': dff['pop'],
        'type': 'bar',
        'marker': marker
    }, 3, 1)
    fig['layout']['showlegend'] = False
    fig['layout']['height'] = 800
    fig['layout']['margin'] = {
        'l': 40,
        'r': 10,
        't': 60,
        'b': 200
    }
    fig['layout']['yaxis3']['type'] = 'log'
    return fig

In [12]:
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

In [ ]:
if __name__ == '__main__':
    app.run_server(host='0.0.0.0',port=int(portID)) # Domino hosts all apps at 0.0.0.0:8888

 * Running on http://0.0.0.0:8883/ (Press CTRL+C to quit)
/home/ubuntu/.local/lib/python3.6/site-packages/dash/resources.py:36: UserWarning:

A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available

127.0.0.1 - - [24/Mar/2022 07:19:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 07:19:02] "GET /_dash-component-suites/dash_table_experiments/dash_table_experiments.css?v=0.6.0 HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 07:19:02] "GET /_dash-component-suites/dash_core_components/react-select@1.0.0-rc.3.min.css?v=0.21.1 HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 07:19:02] "GET /_dash-component-suites/dash_core_components/rc-slider@6.1.2.css?v=0.21.1 HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 07:19:02] "GET /_dash-component-suites/dash_core_components/react-virtualized@9.9.0.css?v=0.21.1 HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2022 07:19:02] "GET /_dash-component-suites/dash_core_components/react-virtualized-select@3.1.0.css?v=0.21.1 HTTP/1.1" 200 -
127.0.0.1 - - [